In [97]:
import operator
from typing import Callable, Tuple

import sympy as sp
from IPython.display import Math
from numpy.random import choice
from sympy import latex
from sympy.abc import x
from random import randint
import matplotlib

In [98]:
y = sp.Function('y')
matplotlib.use("Agg")

In [99]:
%run document_generator.ipynb

In [100]:
def latex_image(path: str) -> str:
    return r"""\begin{figure}[H]
    \centering
    \includegraphics[width=0.5\textwidth]{%s}
    \label{fig:figure}
\end{figure}""" % path

In [101]:
def random_float_coef():
    return sp.S(choice(list(set(range(-3, 4)) - {0}))) / choice([1, 2, 3])


def func_generator_decorator(expr: sp.Expr) -> Callable[[], sp.Expr]:
    def generate_func() -> sp.Expr:
        symbol = list(expr.free_symbols)[0]
        return random_float_coef() * expr.subs(symbol, random_float_coef() * symbol)

    return generate_func


generate_sin = func_generator_decorator(sp.sin(x))
generate_cos = func_generator_decorator(sp.cos(x))
generate_exp = func_generator_decorator(sp.exp(x))


def generate_id() -> sp.Expr:
    return x


def generate_polynom() -> sp.Expr:
    return sum([
        sp.S(choice(list(range(-3, 4)) + i * [0])) / choice([1, 2, 3]) * x ** i for i in range(3)
    ])


def generate_random_func():
    generators = [generate_sin,
                  generate_cos,
                  generate_id,
                  generate_polynom,
                  generate_exp
                  ]
    operators = [operator.add, operator.mul]
    func = None
    while func is None or len(func.free_symbols) < 1:
        func1, func2 = map(lambda generator: generator(), choice(generators, size=2, replace=False))
        random_operator = choice(operators)
        func = random_operator(func1, func2)
    return func

In [102]:
DESCRIPTION = "Решить с помощью метода dsolve задачу Коши. Построить график решения на отрезке $x$ от 0 до 2"

OUT_ANSWER_PATH = "../out/answers/answer18/"

In [103]:
def task_condition(func: sp.Expr, value1: sp.Number, value2: sp.Number, value3: sp.Number) -> str:
    return f"\n\n$$y''' + y'' + y' + y = {latex(func)}," \
           f"\\qquad y(0)={latex(value1)}," \
           f"\\qquad y'(0)={latex(value2)}," \
           f"\\qquad y''(0)={latex(value3)}$$"

In [115]:
def solve_task(option: int, func: sp.Expr, value1: sp.Number, value2: sp.Number, value3: sp.Number) -> str:
    eq = sp.Eq(sum([y(x).diff(x, i) for i in range(4)]), func)
    solution = sp.dsolve(eq, y(x)).rhs
    symbols_dict = {str(symbol): symbol for symbol in solution.free_symbols}
    c1, c2, c3 = list(sp.linsolve(
        [sp.Eq(solution.diff(x, i).subs(x, 0), value) for i, value in enumerate([value1, value2, value3])],
        symbols_dict["C1"], symbols_dict["C2"], symbols_dict["C3"]))[0]
    final_solution = solution.subs({symbols_dict["C1"]: c1, symbols_dict["C2"]: c2, symbols_dict["C3"]: c3})
    p = sp.plot(final_solution, (x, 0, 2), show=False)
    p.save(OUT_ANSWER_PATH + str(option))
    matplotlib.pyplot.close()
    return f"\n\n\\begin{{math}}\n" \
           f"y(x) = {latex(final_solution)}\n" \
           f"\\end{{math}}\n\n" \
           f"{latex_image(OUT_ANSWER_PATH + str(option) + '.png')}"

In [116]:
def generate_task_and_answer(option: int) -> Tuple[str, str]:
    while True:
        try:
            func = generate_random_func()
            values = [randint(-3, 3) for _ in range(3)]
            return task_condition(func, *values), solve_task(option, func, *values)
        except Exception as e:
            print(e)

In [117]:
doc = DocumentGenerator(generate_task_and_answer, DESCRIPTION)

In [118]:
write_tasks_and_solutions(doc, "../out/tasks/task-18.tex", OUT_ANSWER_PATH + "answer-18.tex", 30)